In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

nba_df= pd.read_csv("Resources/shot_logs.csv")


In [2]:
with pd.option_context('display.max_columns', None):
    display(nba_df.head())

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,player_name,player_id
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,1.9,7.7,2,made,"Anderson, Alan",101187,1.3,1,2,brian roberts,203148
1,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,1,0:14,3.4,0,0.8,28.2,3,missed,"Bogdanovic, Bojan",202711,6.1,0,0,brian roberts,203148
2,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,3,1,0:00,NaN,3,2.7,10.1,2,missed,"Bogdanovic, Bojan",202711,0.9,0,0,brian roberts,203148
3,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,4,2,11:47,10.3,2,1.9,17.2,2,missed,"Brown, Markel",203900,3.4,0,0,brian roberts,203148
4,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,5,2,10:34,10.9,2,2.7,3.7,2,missed,"Young, Thaddeus",201152,1.1,0,0,brian roberts,203148


In [3]:
#id unique shooters, and count
nba_shooter= nba_df['player_name'].unique()
shooters= len(nba_shooter)
print(f"We have {shooters} unique shooters in this dataset.")
name_check= nba_df['CLOSEST_DEFENDER_PLAYER_ID'].unique()
defenders=len(name_check)
print(f"We have {defenders} unique defenders in this dataset.")
same_name= nba_df['CLOSEST_DEFENDER'].unique()
same_name_check= len(same_name)
print(f"Comparing the number of pairs of defenders with the same name: {defenders-same_name_check}")

#grouped_def_df=nba_df.groupby("CLOSEST_DEFENDER_PLAYER_ID")
#convert all NaN in Shot_clock category to game_clock
#drop columns
cleaned_df= nba_df.drop(columns=["MATCHUP", "LOCATION", "DRIBBLES", "TOUCH_TIME"])
cleaned_df

We have 281 unique shooters in this dataset.
We have 474 unique defenders in this dataset.
Comparing the number of pairs of defenders with the same name: 1


,GAME_ID,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,player_name,player_id
0,21400899,W,24,1,1,1:09,10.8,7.7,2,made,"Anderson, Alan",101187,1.3,1,2,brian roberts,203148
1,21400899,W,24,2,1,0:14,3.4,28.2,3,missed,"Bogdanovic, Bojan",202711,6.1,0,0,brian roberts,203148
2,21400899,W,24,3,1,0:00,NaN,10.1,2,missed,"Bogdanovic, Bojan",202711,0.9,0,0,brian roberts,203148
3,21400899,W,24,4,2,11:47,10.3,17.2,2,missed,"Brown, Markel",203900,3.4,0,0,brian roberts,203148
4,21400899,W,24,5,2,10:34,10.9,3.7,2,missed,"Young, Thaddeus",201152,1.1,0,0,brian roberts,203148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128064,21400006,L,-16,5,3,1:52,18.3,8.7,2,missed,"Smart, Marcus",203935,0.8,0,0,jarrett jack,101127
128065,21400006,L,-16,6,4,11:28,19.8,0.6,2,made,"Turner, Evan",202323,0.6,1,2,jarrett jack,101127
128066,21400006,L,-16,7,4,11:10,23.0,16.9,2,made,"Thornton, Marcus",201977,4.2,1,2,jarrett jack,101127
128067,21400006,L,-16,8,4,2:37,9.1,18.3,2,missed,"Bradley, Avery",202340,3.0,0,0,jarrett jack,101127


In [ ]:
#convert all NaN in Shot_clock category to game_clock

In [4]:
#rename player_name and player_id to fit all caps style
cleaned_df.rename(columns={"player_name": "PLAYER_NAME",
                  "player_id": "PLAYER_ID"}, inplace=True)

In [5]:
cleaned_df.dtypes

GAME_ID                         int64
W                              object
FINAL_MARGIN                    int64
SHOT_NUMBER                     int64
PERIOD                          int64
GAME_CLOCK                     object
SHOT_CLOCK                    float64
SHOT_DIST                     float64
PTS_TYPE                        int64
SHOT_RESULT                    object
CLOSEST_DEFENDER               object
CLOSEST_DEFENDER_PLAYER_ID      int64
CLOSE_DEF_DIST                float64
FGM                             int64
PTS                             int64
PLAYER_NAME                    object
PLAYER_ID                       int64
dtype: object

In [7]:


to_clean= cleaned_df["GAME_CLOCK"].str.split(":",expand=True).astype(int)
cleaned_df["GAME_CLOCK_SECONDS"]=to_clean[0]*60 + to_clean[1]
counter= 0
cleaned_df["TOTAL_TIME_SECONDS"] = ""
for index,rows in cleaned_df.iterrows(): 
    multiplier= 0
    if rows["PERIOD"] == 1:
        multiplier= 36*60
    elif rows["PERIOD"] == 2:
        multiplier= 24*60
    elif rows["PERIOD"] == 3:
        multiplier= 12*60
    cleaned_df.loc[index,["TOTAL_TIME_SECONDS"]]=multiplier+cleaned_df["GAME_CLOCK_SECONDS"][index]

cleaned_df

,GAME_ID,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,PLAYER_NAME,PLAYER_ID,GAME_CLOCK_SECONDS,TOTAL_TIME_SECONDS
0,21400899,W,24,1,1,1:09,10.8,7.7,2,made,"Anderson, Alan",101187,1.3,1,2,brian roberts,203148,69,2229
1,21400899,W,24,2,1,0:14,3.4,28.2,3,missed,"Bogdanovic, Bojan",202711,6.1,0,0,brian roberts,203148,14,2174
2,21400899,W,24,3,1,0:00,NaN,10.1,2,missed,"Bogdanovic, Bojan",202711,0.9,0,0,brian roberts,203148,0,2160
3,21400899,W,24,4,2,11:47,10.3,17.2,2,missed,"Brown, Markel",203900,3.4,0,0,brian roberts,203148,707,2147
4,21400899,W,24,5,2,10:34,10.9,3.7,2,missed,"Young, Thaddeus",201152,1.1,0,0,brian roberts,203148,634,2074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128064,21400006,L,-16,5,3,1:52,18.3,8.7,2,missed,"Smart, Marcus",203935,0.8,0,0,jarrett jack,101127,112,832
128065,21400006,L,-16,6,4,11:28,19.8,0.6,2,made,"Turner, Evan",202323,0.6,1,2,jarrett jack,101127,688,688
128066,21400006,L,-16,7,4,11:10,23.0,16.9,2,made,"Thornton, Marcus",201977,4.2,1,2,jarrett jack,101127,670,670
128067,21400006,L,-16,8,4,2:37,9.1,18.3,2,missed,"Bradley, Avery",202340,3.0,0,0,jarrett jack,101127,157,157


In [6]:
# cleaned_df.loc[cleaned_df["PERIOD"] == 1,["TOTAL_TIME_SECONDS"]]=(cleaned_df.loc[cleaned_df["PERIOD"] == 1, ["GAME_CLOCK_SECONDS"]]) + (36*60)
# cleaned_df.loc[cleaned_df["PERIOD"] == 2,["TOTAL_TIME_SECONDS"]]=(cleaned_df.loc[cleaned_df["PERIOD"] == 2, ["GAME_CLOCK_SECONDS"]]) + (24*60)
# cleaned_df.loc[cleaned_df["PERIOD"] == 3,["TOTAL_TIME_SECONDS"]]=(cleaned_df.loc[cleaned_df["PERIOD"] == 3, ["GAME_CLOCK_SECONDS"]]) + (12*60)

KeyError: "None of [Index(['GAME_CLOCK_SECONDS'], dtype='object')] are in the [columns]"

In [8]:
to_clean.rename(columns={0: "MINUTES", 1: "SECONDS"}, inplace=True)
# print(to_clean)
#merge into cleaned_df
cleaned_df=pd.concat([cleaned_df, to_clean], axis=1)
# print(new_df)
#if 0 column== 0 AND if column 1 <= 24 then replace
counter=0


In [9]:
cleaned_df["SHOT_CLOCK"].fillna(cleaned_df["SECONDS"], inplace=True)
# for index,rows in cleaned_df.iterrows():
#     if np.isnan(rows["SHOT_CLOCK"]) == True:
#         rows["SHOT_CLOCK"] = cleaned_df["SECONDS"][index]
cleaned_df.head(50)
#cleaned_df.dtypes

,GAME_ID,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,SHOT_DIST,PTS_TYPE,SHOT_RESULT,...,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,PLAYER_NAME,PLAYER_ID,GAME_CLOCK_SECONDS,TOTAL_TIME_SECONDS,MINUTES,SECONDS
0,21400899,W,24,1,1,1:09,10.8,7.7,2,made,...,101187,1.3,1,2,brian roberts,203148,69,2229,1,9
1,21400899,W,24,2,1,0:14,3.4,28.2,3,missed,...,202711,6.1,0,0,brian roberts,203148,14,2174,0,14
2,21400899,W,24,3,1,0:00,0.0,10.1,2,missed,...,202711,0.9,0,0,brian roberts,203148,0,2160,0,0
3,21400899,W,24,4,2,11:47,10.3,17.2,2,missed,...,203900,3.4,0,0,brian roberts,203148,707,2147,11,47
4,21400899,W,24,5,2,10:34,10.9,3.7,2,missed,...,201152,1.1,0,0,brian roberts,203148,634,2074,10,34
5,21400899,W,24,6,2,8:15,9.1,18.4,2,missed,...,101114,2.6,0,0,brian roberts,203148,495,1935,8,15
6,21400899,W,24,7,4,10:15,14.5,20.7,2,missed,...,101127,6.1,0,0,brian roberts,203148,615,615,10,15
7,21400899,W,24,8,4,8:00,3.4,3.5,2,made,...,203486,2.1,1,2,brian roberts,203148,480,480,8,0
8,21400899,W,24,9,4,5:14,12.4,24.6,3,missed,...,202721,7.3,0,0,brian roberts,203148,314,314,5,14
9,21400890,W,1,1,2,11:32,17.4,22.4,3,missed,...,201961,19.8,0,0,brian roberts,203148,692,2132,11,32


In [ ]:
#variable_check


In [ ]:
# for index, rows in new_df.iterrows():
#     if rows['MINUTES'] == 0 and rows['SECONDS'] <= 24:
#         result= rows['SECONDS']
#         if result == int(rows["SHOT_CLOCK"].split(":")[1]):
#             new_df["SHOT_CLOCK"][index] = result
#         print(result)
#         print(rows["SHOT_CLOCK"])
#     counter =+ 1
#     if counter == 5:
#         break
# new_df

In [ ]:
# pd.to_timedelta(new_df["GAME_CLOCK"])
#cleaned_df.set_index()

In [ ]:
#clean game_clock to shot_clock conversion, so it isn't an object type
#convert 0:00 to 0, and 0:12 to 12 etc
#cleaned_df.dtypes
# cleaned_df["SHOT_CLOCK"].astype(float)
# for i in range(25): 
#     shot_clock_range=f"0:{i}"
# .replace?
#     cleaned_df= cleaned_df.replace((cleaned_df["SHOT_CLOCK"] == shot_clock_range), int(i))
# split_shot=cleaned_df["SHOT_CLOCK"].str.split(":").astype(int)
#split_shot[2][1] index for first shot clock value that has a number to convert
# answer=cleaned_df['split_shot'][2]
# print(isinstance(answer, list))
# # print(int(answer[1]))
# counter=0
# for index, rows in cleaned_df.iterrows(): #iterate through dataframe
#     print(rows["split_shot"])
#     try:
#         if (isinstance(rows["split_shot"],list)) == True: #determine if object is list or NaN
#             result=int(rows['split_shot'][index][1]) #give me the second element in list, and convert to int
#             print(result)
#             rows["SHOT_CLOCK"].replace(rows['SHOT_CLOCK'][index],result) #change answer in shot_clock column
#             # to int
#     except: #do nothing
#         counter=+ 1
#         if counter == 5: #use counter of 5 to check first 5 rows
#             break
#         print("NaN, skipping..")
        
# split_shot



In [ ]:
cleaned_df.to_csv("Resources/Final_nba_df.csv", index= False)